In [14]:
import torch
print(torch.cuda.is_available())


False


In [3]:

import json
with open(r"data/QA_model/train_simpleTransformer.json", "r") as read_file:
    train = json.load(read_file)
     

In [4]:
train     

[{'context': 'In this Order and Authorization, we grant, to the extent set forth below, the request of Kuiper Systems LLC (Kuiper or Amazon) to provide satellite services.\n            Operating 3,372 satellites in 102 orbital planes at altitudes of 590 km, 610 km, and 630 km in a circular orbit.\n            At 590 km, 30 orbital planes with 28 satellites per plane for a total of 840 satellites at inclination of 33 degree.\n            At 610 km, 42 orbital planes with 36 satellites per plane for a total of 1512 satellites at inclination of 42 degree.\n            At 630 km, 30 orbital planes with 34 satellites per plane for a total of 1020 satellite at inclination of 51.9 degree.\n            The constellation are require to launch and operate 50 percent of its satellites no later than July 30, 2026, and must launch the remaining space stations necessary to complete its authorized service constellation, place them in their assigned orbits, and operate each of them in accordance with 

In [5]:
with open(r"data/QA_model/validate_simpleTransformer.json", "r") as read_file:
    validation = json.load(read_file)

In [6]:

validation

[{'context': 'Release date: October 29, 1995 In this Order and Authorization, we grant, to the extent set forth below, the request of Ligado Networks LLC to provide Fixed Satellite Services (FSS). Operating 2320 satellites in 58 orbital planes in total at altitudes of 500, 600, 700 and 800 kilometers. At an altitude of 500 km, there are 15 orbital planes, each hosting 36 satellites, resulting in a total of 540 satellites at an inclination of 36.5 degrees. For the 600 km altitude, 23 orbital planes are present, with each plane containing 50 satellites, summing up to 1150 satellites at an inclination of 49 degrees. At the 700 km mark, 15 orbital planes are equipped with 27 satellites each, leading to a total of 405 satellites at a 51.9-degree inclination. Lastly, at 800 km, there are 5 orbital planes, and each has 45 satellites, amounting to a total of 225 satellites at an inclination of 59.3 degrees. The constellation are require to launch and operate 50 percent of its satellites no lat

In [7]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
     

In [8]:

model_type="bert"
model_name= "bert-base-cased"

if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [12]:

### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [13]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

c:\Users\zohar\.conda\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zohar\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: 'use_cuda' set to True when cuda is unavailable. Make sure CUDA is available or set use_cuda=False.

In [ ]:

### Remove output folder
!rm -rf outputs

In [ ]:

# Train the model
model.train_model(train, eval_data=validation)

In [ ]:
# Evaluate the model
result, texts = model.eval_model(validation)

In [ ]:
# 1. Load the .txt file as the context
with open('data/authorize_doc/StarlinkGen2_FCC-22-91A1.txttxt', 'r') as file:
    context = file.read()

In [ ]:
# 2. Define your list of questions
questions = [
    {
        "question": "What's the name of the satellite constellation the company seeks to deploy or operate?",
        "id": "const_name"
    },
    {
        "question": "On which date was the document released?",
        "id": "date_release"
    },
    {
        "question": "By which date must the company launch and operate half of its satellites?",
        "id": "date_50"
    },
    {
        "question": "By which date is the company expected to have all its satellites operational?",
        "id": "date_100"
    },
    {
        "question": "How many satellites is the company authorized to deploy and operate for this constellation?",
        "id": "total_sat_const"
    },
    {
        "question": "At which authorized altitudes will the company deploy its satellites?",
        "id": "altitude"
    },
    {
        "question": "What are the authorized satellite inclinations within the corresponding altitudes?",
        "id": "inclination"
    },
    {
        "question": "How many orbital planes, corresponding to given altitudes and inclinations, has the company been authorized for?",
        "id": "number_orb_plane"
    },
    {
        "question": "How many satellites are allocated to each orbital plane?",
        "id": "total_sat_per_orb_plane"
    },
    {
        "question": "How many satellites, for each altitude and inclination, are there across all matching orbital planes?",
        "id": "total_sat_per_alt_incl"
    },
    {
        "question": "What is the satellite's expected operational lifetime in years?",
        "id": "operational_lifetime"
    }
]

In [ ]:
to_predict = [
    {
        "context": context,
        "qas": questions
    }
]

In [ ]:

answers, probabilities = model.predict(to_predict)

print(answers)